In [47]:
from tensorflow import data, size

from keras import Sequential, losses, metrics, optimizers
from keras.backend import clear_session
from keras.layers import LSTM, InputLayer

import pandas as pd
import numpy as np
import os

import config as c


In [48]:
def csv_to_tensor(target=c.input, step=c.step, take=None):
    sub_dataset = []
    for file in os.listdir(target)[:take]:
        df = pd.read_csv(os.path.join(target, file))

        df = data.Dataset.from_tensor_slices(df)
        df = df.window(step, shift=1, drop_remainder=True).flat_map(
            lambda x: x.batch(c.BATCH_SIZE))
        sub_dataset.append(df)

    tensor = sub_dataset[0]
    for dataset in sub_dataset[1:]:
        tensor = tensor.concatenate(dataset)
    return tensor


In [49]:
take = 10
inputs = csv_to_tensor(target=c.input, take=take)
outputs = csv_to_tensor(target=c.output, take=take)

print('Data imported from:', take, 'videos')


Data imported from: 10 videos


In [50]:
dataset = data.Dataset.zip((inputs, outputs))

train_dataset = dataset.take(4000).shuffle(c.BUFFER_SIZE).batch(
    c.BATCH_SIZE).prefetch(data.experimental.AUTOTUNE)
val_dataset = dataset.skip(4000).batch(
    c.BATCH_SIZE).prefetch(data.experimental.AUTOTUNE)


In [57]:
clear_session()

model = Sequential(name='food_frame_classification')
model.add(InputLayer(input_shape=(None, c.features)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(4, return_sequences=True, activation='relu'))
model.add(LSTM(1, return_sequences=True, activation='relu'))

model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=optimizers.Adam(),
              metrics=metrics.Accuracy('val_accuracy'),
              run_eagerly=True)

model.summary()

Model: "food_frame_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 64)          147712    
                                                                 
 lstm_1 (LSTM)               (None, None, 4)           1104      
                                                                 
 lstm_2 (LSTM)               (None, None, 1)           24        
                                                                 
Total params: 148,840
Trainable params: 148,840
Non-trainable params: 0
_________________________________________________________________


In [58]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=100)


Epoch 1/100
63/63 [==============================] - 8s 114ms/step - loss: 0.6886 - val_accuracy: 0.6393 - val_loss: 0.7201 - val_val_accuracy: 0.5600
Epoch 2/100
63/63 [==============================] - 7s 110ms/step - loss: 0.6473 - val_accuracy: 0.5035 - val_loss: 0.7386 - val_val_accuracy: 0.5188
Epoch 3/100
63/63 [==============================] - 7s 106ms/step - loss: 0.6071 - val_accuracy: 0.4999 - val_loss: 0.8527 - val_val_accuracy: 0.5172
Epoch 4/100
63/63 [==============================] - 8s 115ms/step - loss: 0.5792 - val_accuracy: 0.5312 - val_loss: 0.8217 - val_val_accuracy: 0.5668
Epoch 5/100
63/63 [==============================] - 7s 109ms/step - loss: 0.5644 - val_accuracy: 0.5732 - val_loss: 1.0712 - val_val_accuracy: 0.5539
Epoch 6/100
63/63 [==============================] - 8s 116ms/step - loss: 0.5508 - val_accuracy: 0.5940 - val_loss: 0.9813 - val_val_accuracy: 0.6181
Epoch 7/100
63/63 [==============================] - 8s 114ms/step - loss: 0.5421 - val_accura

In [53]:
model.save('saved_model')

INFO:tensorflow:Assets written to: saved_model\assets


In [54]:
predict, _ = csv_to_tensor(target=c.input, take=1)
len_p = list(predict)
print(len_p)
predict = predict.batch(len_p).batch(1)


ValueError: too many values to unpack (expected 2)

In [ ]:
result = model.predict(predict)
print(int(np.round(result)))


ValueError: Input 0 of layer "food_frame_classification" is incompatible with the layer: expected shape=(None, None, 512), found shape=(1, 1, 5, 512)